In [1]:
import sys
import torch
sys.path.append("/home/student/andrewheschl/Documents/ClassificationPipeline")
from src.json_models.src.model_generator import ModelGenerator

In [ ]:
import glob
files = glob.glob('/home/andrew.heschl/Documents/ClassificationPipeline/models/run/*.json')

In [2]:
for file in files:
    gen = ModelGenerator(json_path="/home/andrewheschl/PycharmProjects/ClassificationPipeline/models/efficientnets/efficientnet.json")

In [3]:
model = gen.get_model()
print(gen.get_log_kwargs())

{'backbone': 'EfficientNetB0', 'conv': 'DW', 'order': 'NA', 'mode': 'NA'}


In [4]:
import torch
from src.json_models.src.mobile_net.mobilenetv2 import MobileNetV2

data = torch.ones((100, 3, 512, 512))
out = model(data)
out.shape

torch.Size([100, 2, 1, 1]) torch.Size([100, 56, 256, 256])


RuntimeError: The size of tensor a (56) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
model